In [2]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
!pip install aif360

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.7/259.7 kB 5.8 MB/s eta 0:00:0000:01


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision

import timm

import matplotlib.pyplot as plt
import sys
from tqdm.notebook import tqdm

import os
import time

import numpy as np
import pandas as pd

import torch

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from torchvision import datasets
from torchvision import transforms

from PIL import Image

In [5]:
#Dynamically set device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

GRAYSCALE = False

Using device: cuda


# Model - Fine-tuned on CelebA

In [6]:
import torchvision.models as models

model = models.resnet18()

#Modify the final fully connected layer to output two classes
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2) 

model_path = "/kaggle/input/resnet18_baseline_celeba/pytorch/default/1/fine_tuned_best_ever_resnet18"
model.load_state_dict(torch.load(model_path, weights_only = True))


for param in model.parameters():
    param.requires_grad = False

#last layer trainable
for param in model.fc.parameters():
        param.requires_grad = True

In [7]:
#criterion, optimizer, and other parameters
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 5
validate_every = 2
patience = 3  #Number of epochs to wait before early stopping

# Loading Dataset in train loaders

In [8]:
#The target label is attractiveness and protected attribute is gender.

data_root = "/kaggle/input/celeba-dataset/"
label_attr = "Attractive"
protected_attr = "Male"

In [9]:
class CelebaDataset(Dataset):
    
    def __init__(self, csv_path, img_dir, transform=None, label_attr = 'Attractive', protected_attr = 'Male'):
        
        df = pd.read_csv(csv_path, index_col=0)
        self.img_dir = img_dir
        self.csv_path = csv_path
        self.img_names = df.index.values
        self.y = df[label_attr].values
        self.p = df[protected_attr].values
        self.transform = transform
        
    def __getitem__(self, index):
        img = Image.open(os.path.join(self.img_dir,
                      self.img_names[index]))

        if self.transform is not None:
            img = self.transform(img)
            
        label = self.y[index]
        protected = self.p[index]
        return img, label, protected
        
    def __len__(self):
        return self.y.shape[0]


In [10]:
other_data_root = '/kaggle/working/' #for writing purposes

In [11]:
def prepare_dataset(label_attr = 'Attractive', protected_attr = 'Male'):

    print('Using label attribute:', label_attr)

    df1 = pd.read_csv(data_root + 'list_attr_celeba.csv',usecols=[label_attr, protected_attr])
    #Make 0 (female) & 1 (male) labels instead of -1 & 1
    df1.loc[df1[label_attr] == -1, label_attr] = 0
    df1.loc[df1[protected_attr] == -1, protected_attr] = 0
    

    df2 = pd.read_csv(data_root + 'list_eval_partition.csv')
    df2.columns = ['image_id', 'partition']
    image_series = df2['image_id']
    df2 = df2.set_index('image_id')

    df1 = df1.set_index(image_series)

    df3 = pd.concat([df1,df2],axis=1)
    
    df3.to_csv(other_data_root + 'celeba-attractive-partitions.csv')

    df4 = pd.read_csv(other_data_root + 'celeba-attractive-partitions.csv', index_col=0)

    df4.loc[df4['partition'] == 0].to_csv(other_data_root + 'celeba-attractive-train.csv')
    df4.loc[df4['partition'] == 1].to_csv(other_data_root + 'celeba-attractive-valid.csv')
    df4.loc[df4['partition'] == 2].to_csv(other_data_root + 'celeba-attractive-test.csv')
    
def get_loaders():

    prepare_dataset(label_attr, protected_attr)
    
    #Note that transforms.ToTensor()
    #already divides pixels by 255. internally

    custom_transform = transforms.Compose([transforms.CenterCrop((178, 178)),
                                           transforms.Resize((128, 128)),
                                           transforms.ToTensor()])

    train_dataset = CelebaDataset(csv_path=other_data_root + 'celeba-attractive-train.csv',
                                  img_dir=data_root + 'img_align_celeba/img_align_celeba',
                                  transform=custom_transform,
                                  label_attr=label_attr,
                                  protected_attr = protected_attr)

    valid_dataset = CelebaDataset(csv_path=other_data_root + 'celeba-attractive-valid.csv',
                                  img_dir=data_root + 'img_align_celeba/img_align_celeba',
                                  transform=custom_transform,
                                  label_attr=label_attr,
                                  protected_attr = protected_attr)

    test_dataset = CelebaDataset(csv_path=other_data_root + 'celeba-attractive-test.csv',
                                 img_dir=data_root + 'img_align_celeba/img_align_celeba',
                                 transform=custom_transform,
                                 label_attr=label_attr,
                                 protected_attr = protected_attr)

    train_loader = DataLoader(dataset=train_dataset,
                              batch_size=BATCH_SIZE,
                              shuffle=True,
                              num_workers=4)

    valid_loader = DataLoader(dataset=valid_dataset,
                              batch_size=BATCH_SIZE,
                              shuffle=False,
                              num_workers=4)
    
    test_loader = DataLoader(dataset=test_dataset,
                             batch_size=BATCH_SIZE,
                             shuffle=False,
                             num_workers=4)

    return train_loader, valid_loader, test_loader


In [12]:
BATCH_SIZE = 256

In [13]:
train_loader, valid_loader, test_loader = get_loaders()

Using label attribute: Attractive


In [14]:
print(f"The number of train batches : {len(train_loader)}")
print(f"The number of validation batches : {len(valid_loader)}")
print(f"The number of test batches : {len(test_loader)}")

The number of train batches : 636
The number of validation batches : 78
The number of test batches : 78


# Basic Testing

In [15]:
def test_model(model_in_fn, test_loader, gpu_yes = True):
    model_in_fn.eval()  # Set model to evaluation mode
    if gpu_yes:
        device = 'cuda'
        model_in_fn.to(device)
    else:
        device='cpu'
        model_in_fn.to('cpu')
    # Initialize counters
    correct, total = 0, 0
    male_correct, female_correct = 0, 0
    male_total, female_total = 0, 0
    
    # Initialize confusion matrix components for males and females
    male_tp, male_fp, male_tn, male_fn = 0, 0, 0, 0
    female_tp, female_fp, female_tn, female_fn = 0, 0, 0, 0
    all_ones, all_zeros = 0, 0
    
    with torch.no_grad():
        for images, labels, genders in test_loader:
            images, labels, genders = images.to(device), labels.to(device), genders.to(device)
            outputs = model_in_fn(images)
            _, predicted = torch.max(outputs, 1)
            
            # Update overall accuracy
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            
            # Count total positive and negative cases
            all_ones += (labels == 1).sum().item()
            all_zeros += (labels == 0).sum().item()
            
            # Separate male and female predictions
            male_mask = (genders == 1)
            female_mask = (genders == 0)
            
            # Update accuracy counts
            male_correct += (predicted[male_mask] == labels[male_mask]).sum().item()
            female_correct += (predicted[female_mask] == labels[female_mask]).sum().item()
            male_total += male_mask.sum().item()
            female_total += female_mask.sum().item()
            
            # Update confusion matrix components for males
            male_tp += ((predicted[male_mask] == 1) & (labels[male_mask] == 1)).sum().item()
            male_fp += ((predicted[male_mask] == 1) & (labels[male_mask] == 0)).sum().item()
            male_tn += ((predicted[male_mask] == 0) & (labels[male_mask] == 0)).sum().item()
            male_fn += ((predicted[male_mask] == 0) & (labels[male_mask] == 1)).sum().item()
            
            # Update confusion matrix components for females
            female_tp += ((predicted[female_mask] == 1) & (labels[female_mask] == 1)).sum().item()
            female_fp += ((predicted[female_mask] == 1) & (labels[female_mask] == 0)).sum().item()
            female_tn += ((predicted[female_mask] == 0) & (labels[female_mask] == 0)).sum().item()
            female_fn += ((predicted[female_mask] == 0) & (labels[female_mask] == 1)).sum().item()
    
    # Calculate metrics
    overall_accuracy = correct / total
    male_accuracy = male_correct / male_total if male_total > 0 else 0
    female_accuracy = female_correct / female_total if female_total > 0 else 0
    
    # Calculate TPR and FPR correctly
    male_tpr = male_tp / (male_tp + male_fn) if (male_tp + male_fn) > 0 else 0
    female_tpr = female_tp / (female_tp + female_fn) if (female_tp + female_fn) > 0 else 0
    male_fpr = male_fp / (male_fp + male_tn) if (male_fp + male_tn) > 0 else 0
    female_fpr = female_fp / (female_fp + female_tn) if (female_fp + female_tn) > 0 else 0
    
    # Fairness metrics
    DEO = abs(male_tpr - female_tpr)
    FPR_diff = abs(male_fpr - female_fpr)
    
    print(f"Overall Accuracy: {overall_accuracy:.4f}")
    print(f"Male Accuracy: {male_accuracy:.4f}")
    print(f"Female Accuracy: {female_accuracy:.2f}")
    print(f"Male TPR: {male_tpr:.2f}")
    print(f"Female TPR: {female_tpr:.2f}")
    print(f"Male FPR: {male_fpr:.2f}")
    print(f"Female FPR: {female_fpr:.2f}")
    print(f"Demographic Equal Opportunity (DEO): {DEO:.2f}")
    print(f"False Positive Rate Difference (FPR): {FPR_diff:.2f}")
    
    return {
        "overall_accuracy": overall_accuracy,
        "male_accuracy": male_accuracy,
        "female_accuracy": female_accuracy,
        "male_tpr": male_tpr,
        "female_tpr": female_tpr,
        "male_fpr": male_fpr,
        "female_fpr": female_fpr,
        "DEO": DEO,
        "FPR_difference": FPR_diff,
        "Attractive yes in test": all_ones/total,
        "Attractive no in test": all_zeros/total,
        "Batch Size": BATCH_SIZE
    }, model_in_fn

In [16]:
test_model(model, test_loader)

Overall Accuracy: 0.8006
Male Accuracy: 0.8161
Female Accuracy: 0.79
Male TPR: 0.65
Female TPR: 0.86
Male FPR: 0.13
Female FPR: 0.33
Demographic Equal Opportunity (DEO): 0.21
False Positive Rate Difference (FPR): 0.20


({'overall_accuracy': 0.8005710850616171,
  'male_accuracy': 0.8160725858716785,
  'female_accuracy': 0.7908059116518331,
  'male_tpr': 0.6504702194357367,
  'female_tpr': 0.8567134268537074,
  'male_fpr': 0.12928805378383038,
  'female_fpr': 0.33262960356556415,
  'DEO': 0.20624320741797075,
  'FPR_difference': 0.20334154978173377,
  'Attractive yes in test': 0.49584209998998097,
  'Attractive no in test': 0.504157900010019,
  'Batch Size': 256},
 ResNet(
   (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
   (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (relu): ReLU(inplace=True)
   (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
   (layer1): Sequential(
     (0): BasicBlock(
       (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      

# calibrated_eq_odds_postprocessing - code from IBM F 360

In [17]:
# Original work Copyright (c) 2017 Geoff Pleiss
#
# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.
#
# Modified work Copyright 2018 IBM Corporation
#
# Licensed under the Apache License, Version 2.0 (the "License"); you may not
# use this file except in compliance with the License. You may obtain a copy of
# the License at http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software distributed
# under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR
# CONDITIONS OF ANY KIND, either express or implied. See the License for the
# specific language governing permissions and limitations under the License.
import numpy as np

from aif360.algorithms import Transformer
from aif360.metrics import ClassificationMetric, utils

In [18]:
class CalibratedEqOddsPostprocessing(Transformer):
    """Calibrated equalized odds postprocessing is a post-processing technique
    that optimizes over calibrated classifier score outputs to find
    probabilities with which to change output labels with an equalized odds
    objective [7]_.

    References:
        .. [7] G. Pleiss, M. Raghavan, F. Wu, J. Kleinberg, and
           K. Q. Weinberger, "On Fairness and Calibration," Conference on Neural
           Information Processing Systems, 2017

    Adapted from:
    https://github.com/gpleiss/equalized_odds_and_calibration/blob/master/calib_eq_odds.py
    """

    def __init__(self, unprivileged_groups, privileged_groups,
                 cost_constraint='weighted', seed=None):
        """
        Args:
            unprivileged_groups (dict or list(dict)): Representation for
                unprivileged group.
            privileged_groups (dict or list(dict)): Representation for
                privileged group.
            cost_contraint: fpr, fnr or weighted
            seed (int, optional): Seed to make `predict` repeatable.
        """
        super(CalibratedEqOddsPostprocessing, self).__init__(
            unprivileged_groups=unprivileged_groups,
            privileged_groups=privileged_groups,
            seed=seed)

        self.seed = seed
        self.model_params = None
        self.unprivileged_groups = [unprivileged_groups] \
            if isinstance(unprivileged_groups, dict) else unprivileged_groups
        self.privileged_groups = [privileged_groups] \
            if isinstance(privileged_groups, dict) else privileged_groups
        self.cost_constraint = cost_constraint
        if self.cost_constraint == 'fnr':
            self.fn_rate = 1
            self.fp_rate = 0
        elif self.cost_constraint == 'fpr':
            self.fn_rate = 0
            self.fp_rate = 1
        elif self.cost_constraint == 'weighted':
            self.fn_rate = 1
            self.fp_rate = 1

        self.base_rate_priv = 0.0
        self.base_rate_unpriv = 0.0

    def fit(self, dataset_true, dataset_pred):
        """Compute parameters for equalizing generalized odds using true and
        predicted scores, while preserving calibration.

        Args:
            dataset_true (BinaryLabelDataset): Dataset containing true `labels`.
            dataset_pred (BinaryLabelDataset): Dataset containing predicted
                `scores`.

        Returns:
            CalibratedEqOddsPostprocessing: Returns self.
        """

        # Create boolean conditioning vectors for protected groups
        cond_vec_priv = utils.compute_boolean_conditioning_vector(
            dataset_pred.protected_attributes,
            dataset_pred.protected_attribute_names,
            self.privileged_groups)
        cond_vec_unpriv = utils.compute_boolean_conditioning_vector(
            dataset_pred.protected_attributes,
            dataset_pred.protected_attribute_names,
            self.unprivileged_groups)

        cm = ClassificationMetric(dataset_true, dataset_pred,
                                  unprivileged_groups=self.unprivileged_groups,
                                  privileged_groups=self.privileged_groups)
        self.base_rate_priv = cm.base_rate(privileged=True)
        self.base_rate_unpriv = cm.base_rate(privileged=False)

        # Create a dataset with "trivial" predictions
        dataset_trivial = dataset_pred.copy(deepcopy=True)
        dataset_trivial.scores[cond_vec_priv] = cm.base_rate(privileged=True)
        dataset_trivial.scores[cond_vec_unpriv] = cm.base_rate(privileged=False)
        cm_triv = ClassificationMetric(dataset_true, dataset_trivial,
            unprivileged_groups=self.unprivileged_groups,
            privileged_groups=self.privileged_groups)

        if self.fn_rate == 0:
            priv_cost = cm.generalized_false_positive_rate(privileged=True)
            unpriv_cost = cm.generalized_false_positive_rate(privileged=False)
            priv_trivial_cost = cm_triv.generalized_false_positive_rate(privileged=True)
            unpriv_trivial_cost = cm_triv.generalized_false_positive_rate(privileged=False)

        elif self.fp_rate == 0:
            priv_cost = cm.generalized_false_negative_rate(privileged=True)
            unpriv_cost = cm.generalized_false_negative_rate(privileged=False)
            priv_trivial_cost = cm_triv.generalized_false_negative_rate(privileged=True)
            unpriv_trivial_cost = cm_triv.generalized_false_negative_rate(privileged=False)

        else:
            priv_cost = weighted_cost(self.fp_rate, self.fn_rate, cm, privileged=True)
            unpriv_cost = weighted_cost(self.fp_rate, self.fn_rate, cm, privileged=False)
            priv_trivial_cost = weighted_cost(self.fp_rate, self.fn_rate, cm_triv, privileged=True)
            unpriv_trivial_cost = weighted_cost(self.fp_rate, self.fn_rate, cm_triv, privileged=False)

        unpriv_costs_more = unpriv_cost > priv_cost
        self.priv_mix_rate = (unpriv_cost - priv_cost) / (priv_trivial_cost - priv_cost) if unpriv_costs_more else 0
        self.unpriv_mix_rate = 0 if unpriv_costs_more else (priv_cost - unpriv_cost) / (unpriv_trivial_cost - unpriv_cost)

        return self

    def predict(self, dataset, threshold=0.5):
        """Perturb the predicted scores to obtain new labels that satisfy
        equalized odds constraints, while preserving calibration.

        Args:
            dataset (BinaryLabelDataset): Dataset containing `scores` that needs
                to be transformed.
            threshold (float): Threshold for converting `scores` to `labels`.
                Values greater than or equal to this threshold are predicted to
                be the `favorable_label`. Default is 0.5.
        Returns:
            dataset (BinaryLabelDataset): transformed dataset.
        """
        if self.seed is not None:
            np.random.seed(self.seed)

        cond_vec_priv = utils.compute_boolean_conditioning_vector(
            dataset.protected_attributes,
            dataset.protected_attribute_names,
            self.privileged_groups)
        cond_vec_unpriv = utils.compute_boolean_conditioning_vector(
            dataset.protected_attributes,
            dataset.protected_attribute_names,
            self.unprivileged_groups)

        unpriv_indices = (np.random.random(sum(cond_vec_unpriv))
                       <= self.unpriv_mix_rate)
        unpriv_new_pred = dataset.scores[cond_vec_unpriv].copy()
        unpriv_new_pred[unpriv_indices] = self.base_rate_unpriv

        priv_indices = (np.random.random(sum(cond_vec_priv))
                     <= self.priv_mix_rate)
        priv_new_pred = dataset.scores[cond_vec_priv].copy()
        priv_new_pred[priv_indices] = self.base_rate_priv

        dataset_new = dataset.copy(deepcopy=True)

        dataset_new.scores = np.zeros_like(dataset.scores, dtype=np.float64)
        dataset_new.scores[cond_vec_priv] = priv_new_pred
        dataset_new.scores[cond_vec_unpriv] = unpriv_new_pred

        # Create labels from scores using a default threshold
        dataset_new.labels = np.where(dataset_new.scores >= threshold,
                                      dataset_new.favorable_label,
                                      dataset_new.unfavorable_label)
        return dataset_new

    def fit_predict(self, dataset_true, dataset_pred, threshold=0.5):
        """fit and predict methods sequentially."""
        return self.fit(dataset_true, dataset_pred).predict(
            dataset_pred, threshold=threshold)

######### SUPPORTING FUNCTIONS ##########

def weighted_cost(fp_rate, fn_rate, cm, privileged):
    norm_const = float(fp_rate + fn_rate) if\
                      (fp_rate != 0 and fn_rate != 0) else 1
    return ((fp_rate / norm_const
            * cm.generalized_false_positive_rate(privileged=privileged)
            * (1 - cm.base_rate(privileged=privileged))) +
           (fn_rate / norm_const
            * cm.generalized_false_negative_rate(privileged=privileged)
            * cm.base_rate(privileged=privileged)))

In [19]:
from sklearn.metrics import confusion_matrix

In [26]:
class CalibratedEqOdds:
    def __init__(self):
        self.thresholds = {}

    def fit(self, y_true, y_pred, p_attr):
        cm = confusion_matrix
        tprs, fprs = {}, {}
        
        for group in torch.unique(p_attr):
            group_mask = (p_attr == group)
            y_true_group = y_true[group_mask]
            y_pred_group = y_pred[group_mask]
            tn, fp, fn, tp = cm(y_true_group, y_pred_group).ravel()
            tprs[group.item()] = tp / (tp + fn)
            fprs[group.item()] = fp / (fp + tn)

        avg_tpr = sum(tprs.values()) / len(tprs)
        avg_fpr = sum(fprs.values()) / len(fprs)
        self.thresholds = {group: (avg_tpr, avg_fpr) for group in tprs.keys()}

    def predict(self, y_prob, p_attr):
        y_adjusted = y_prob.clone()
        for i, prob in enumerate(y_prob):
            group = p_attr[i].item()
            tpr, fpr = self.thresholds[group]
            #Adjust probabilities (example heuristic for postprocessing)
            if prob > tpr:
                y_adjusted[i] = 1
            elif prob < fpr:
                y_adjusted[i] = 0
        return y_adjusted

#Integrate postprocessing into testing pipeline
def test_model_with_postprocessing(model, test_loader, postprocessor, gpu_yes=True):
    model.eval()  
    if gpu_yes:
        device = 'cuda'
        model.to(device)
    else:
        device = 'cpu'
        model.to(device)
    
    #Storage for calibration
    y_true_list, y_pred_list, p_attr_list = [], [], []

    #other metric calculations
    correct, total = 0, 0
    male_correct, female_correct = 0, 0
    male_total, female_total = 0, 0
    
    #Initialize confusion matrix components for males and females
    male_tp, male_fp, male_tn, male_fn = 0, 0, 0, 0
    female_tp, female_fp, female_tn, female_fn = 0, 0, 0, 0
    all_ones, all_zeros = 0, 0
    
    with torch.no_grad():
        for images, labels, genders in test_loader:
            images, labels, genders = images.to(device), labels.to(device), genders.to(device)
            outputs = model(images)
            probabilities = torch.softmax(outputs, dim=1)[:, 1]  #Probabilities for class 1
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            
            #Count total positive and negative cases
            all_ones += (labels == 1).sum().item()
            all_zeros += (labels == 0).sum().item()
            
            #Separate male and female predictions
            male_mask = (genders == 1)
            female_mask = (genders == 0)
            
            #Update accuracy counts
            male_correct += (predicted[male_mask] == labels[male_mask]).sum().item()
            female_correct += (predicted[female_mask] == labels[female_mask]).sum().item()
            male_total += male_mask.sum().item()
            female_total += female_mask.sum().item()
            
            #Update confusion matrix components for males
            male_tp += ((predicted[male_mask] == 1) & (labels[male_mask] == 1)).sum().item()
            male_fp += ((predicted[male_mask] == 1) & (labels[male_mask] == 0)).sum().item()
            male_tn += ((predicted[male_mask] == 0) & (labels[male_mask] == 0)).sum().item()
            male_fn += ((predicted[male_mask] == 0) & (labels[male_mask] == 1)).sum().item()
            
            #Update confusion matrix components for females
            female_tp += ((predicted[female_mask] == 1) & (labels[female_mask] == 1)).sum().item()
            female_fp += ((predicted[female_mask] == 1) & (labels[female_mask] == 0)).sum().item()
            female_tn += ((predicted[female_mask] == 0) & (labels[female_mask] == 0)).sum().item()
            female_fn += ((predicted[female_mask] == 0) & (labels[female_mask] == 1)).sum().item()

            
            #Store for calibration
            y_true_list.extend(labels.cpu().numpy())
            y_pred_list.extend(predicted.cpu().numpy())
            p_attr_list.extend(genders.cpu().numpy())
    
    #Convert to tensors for postprocessing
    y_true_tensor = torch.tensor(y_true_list)
    y_pred_tensor = torch.tensor(y_pred_list)
    p_attr_tensor = torch.tensor(p_attr_list)

    #Calculate metrics
    overall_accuracy = correct / total
    male_accuracy = male_correct / male_total if male_total > 0 else 0
    female_accuracy = female_correct / female_total if female_total > 0 else 0
    
    #Calculate TPR and FPR correctly
    male_tpr = male_tp / (male_tp + male_fn) if (male_tp + male_fn) > 0 else 0
    female_tpr = female_tp / (female_tp + female_fn) if (female_tp + female_fn) > 0 else 0
    male_fpr = male_fp / (male_fp + male_tn) if (male_fp + male_tn) > 0 else 0
    female_fpr = female_fp / (female_fp + female_tn) if (female_fp + female_tn) > 0 else 0
    
    #Fairness metrics
    DEO = abs(male_tpr - female_tpr)
    FPR_diff = abs(male_fpr - female_fpr)
    
    print(f"Overall Accuracy: {overall_accuracy:.4f}")
    print(f"Male Accuracy: {male_accuracy:.4f}")
    print(f"Female Accuracy: {female_accuracy:.4f}")
    print(f"Male TPR: {male_tpr:.2f}")
    print(f"Female TPR: {female_tpr:.2f}")
    print(f"Male FPR: {male_fpr:.2f}")
    print(f"Female FPR: {female_fpr:.2f}")
    print(f"Equal Opportunity (EO): {DEO:.2f}")
    print(f"False Positive Rate Difference (FPR): {FPR_diff:.2f}")
    
    # Fit the postprocessor
    postprocessor.fit(y_true_tensor, y_pred_tensor, p_attr_tensor)

    # Adjust predictions using calibrated equalized odds
    calibrated_preds = postprocessor.predict(y_pred_tensor, p_attr_tensor)

    # Recompute metrics with adjusted predictions
    overall_accuracy = (calibrated_preds == y_true_tensor).sum().item() / len(y_true_tensor)
    print(f"Overall Accuracy (Postprocessed): {overall_accuracy:.2f}")

    return overall_accuracy

# Load your dataset and dataloaders
train_loader, valid_loader, test_loader = get_loaders()

# Instantiate the postprocessor
postprocessor = CalibratedEqOdds()

# Test the model with postprocessing
test_model_with_postprocessing(model, test_loader, postprocessor)


Using label attribute: Attractive


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Overall Accuracy: 0.8006
Male Accuracy: 0.8161
Female Accuracy: 0.7908
Male TPR: 0.65
Female TPR: 0.86
Male FPR: 0.13
Female FPR: 0.33
Equal Opportunity (EO): 0.21
False Positive Rate Difference (FPR): 0.20
Overall Accuracy (Postprocessed): 0.80


0.8005710850616171

# Quantization

In [21]:
from torch.utils.data import DataLoader, Subset
import copy

In [23]:
num_calibration_samples = 1000  
calibration_indices = np.random.choice(len(test_loader.dataset), num_calibration_samples, replace=False)
calibration_subset = Subset(test_loader.dataset, calibration_indices)
calibration_dataloader = DataLoader(calibration_subset, batch_size=test_loader.batch_size, shuffle=False)

#Clone the model to avoid modifying the original during quantization
qmodel = models.quantization.resnet18()
num_features = qmodel.fc.in_features
qmodel.fc = nn.Linear(num_features, 2)

#Load the pretrained weights and make a safe copy of the model
model_path = "/kaggle/input/resnet18_baseline_celeba/pytorch/default/1/fine_tuned_best_ever_resnet18"
qmodel.load_state_dict(torch.load(model_path, weights_only = True))
qmodel = copy.deepcopy(qmodel)
qmodel.eval()
qmodel.to('cpu')

#Quantization steps
qmodel.qconfig = torch.quantization.get_default_qconfig('fbgemm')
qmodel_prepared = torch.quantization.prepare(copy.deepcopy(qmodel))
qmodel_prepared.to('cpu')

#Calibration
print("Caliberation results\n")

dictionary_results_calibration, _ = test_model(qmodel_prepared, calibration_dataloader)
qmodel_prepared.to('cpu')

# # Convert to quantized model
# qmodel_int8 = models.quantization.resnet18()
# num_features = qmodel_int8.fc.in_features
# qmodel_int8.fc = nn.Linear(num_features, 2)
# qmodel_int8.to('cpu')

qmodel_int8 = torch.quantization.convert(qmodel_prepared, inplace=True)

# # Testing the quantized model
# dict_, qmodel_int8 = test_model(qmodel_int8, test_loader,False)
# dict_

/opt/conda/lib/python3.10/site-packages/torch/ao/quantization/observer.py:221: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


Caliberation results

Overall Accuracy: 0.8010
Male Accuracy: 0.8133
Female Accuracy: 0.79
Male TPR: 0.68
Female TPR: 0.84
Male FPR: 0.14
Female FPR: 0.30
Demographic Equal Opportunity (DEO): 0.17
False Positive Rate Difference (FPR): 0.16


In [ ]:
# qmodel_int8.to("cuda")

In [24]:
train_loader, valid_loader, test_loader = get_loaders()

Using label attribute: Attractive


In [25]:
postprocessor_ = CalibratedEqOdds()

#Test the model with postprocessing
test_model_with_postprocessing(qmodel_int8, test_loader, postprocessor_,gpu_yes = False )

/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Overall Accuracy: 0.7962
Male Accuracy: 0.8215
Female Accuracy: 0.7802
Male TPR: 0.61
Female TPR: 0.81
Male FPR: 0.11
Female FPR: 0.28
Equal Opportunity (EO): 0.21
False Positive Rate Difference (FPR): 0.18
Overall Accuracy (Postprocessed): 0.80


0.7961627091473801